In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import scipy.stats as stats
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from catboost import CatBoostRegressor, Pool

In [4]:
df = pd.read_csv('Student_performance_data _.csv')
print(df.shape[0])
y = df['GPA'].to_frame()
x = df.iloc[:,:11]
#x = sm.add_constant(x)

2392


In [6]:
# 计算每个变量的 VIF 值
vif_data = pd.DataFrame()
vif_data["Variable"] = x.columns
vif_data["VIF"] = [variance_inflation_factor(x.values, i) for i in range(x.shape[1])]

# 输出 VIF
print(vif_data)

             Variable       VIF
0              Gender  1.935943
1           Ethnicity  1.668854
2   ParentalEducation  3.379635
3     StudyTimeWeekly  3.353247
4            Absences  3.338364
5            Tutoring  1.393269
6     ParentalSupport  3.669053
7     Extracurricular  1.552757
8              Sports  1.406747
9               Music  1.238985
10       Volunteering  1.172704


# 线性回归

In [9]:
class LinReg:
    def __init__(self,dataFn,iy,iXs,train_test = False):
        try:
            self.df = pd.read_csv(dataFn)
        except FileNotFoundError:
            raise FileNotFoundError(f"File {dataFn} not found.")
        except Exception as e:
            raise ValueError(f"An error occurred while loading the data: {e}")
        self.nobs = self.df.shape[0]
        
        try:
            if iy in iXs:
                raise ValueError("LHS variable found in the RHS list.")
            
            if not isinstance(iy, (int, str)):
                raise TypeError("iy should be either an integer or string representing the dependent variable.")

            if isinstance(iy, int):
                iy = [iy]
        except Exception as e:
            raise ValueError(f"An error occurred during match x and y: {e}")

        self.X = self.df.iloc[:,list(iXs)]
        self.y = self.df.iloc[:,list(iy)].squeeze()
        self.train_test = train_test
        self.iXs = iXs
        self.iy = iy
        if self.train_test:
            np.random.seed(42)
            indices = np.random.permutation(self.nobs)
            n_train = int(0.7 * self.nobs)  # 70% for training
            train_indices = indices[:n_train]
            test_indices = indices[n_train:]
                
            self.X_train = self.X.iloc[train_indices]
            self.y_train = self.y.iloc[train_indices]
            self.X_test = self.X.iloc[test_indices]
            self.y_test = self.y.iloc[test_indices]
                

    def getYhat(self, X, betas):
        if X.shape[1] != len(betas):
            raise ValueError("The number of coefficients does not match the number of regressors.")
        
        return X.dot(betas)

    def fit(self):
        try:
            if self.train_test:
                y = self.y_train
                X1 = self.X_train
            else:
                y = self.y
                X1 = self.X
            self.ns = X1.shape[0]
            
            self.RHS = list(self.df.columns[list(self.iXs)])
            self.LHS = list(self.df.columns[list(self.iy)])
    
            X1 = np.hstack([np.ones((self.ns, 1)), X1])  # Add intercept term
            self.RHS = ['Intercept'] + self.RHS
                
            self.nRHS = len(self.RHS) - 1
    
                # QR分解
            Q, R = np.linalg.qr(X1)
            self.Coef = np.linalg.solve(R, Q.T.dot(y)).tolist()
    
            self.yhat = self.getYhat(pd.DataFrame(X1, columns=self.RHS), self.Coef)
            self.ymean = np.mean(y)
            self.SST = (y - self.ymean).dot(y - self.ymean)
            self.SSE = (self.yhat - self.ymean).dot(self.yhat - self.ymean)
            self.R2 = self.SSE / self.SST
            self.adjR2 = 1 - (1 - self.R2) * (self.ns - 1) / (self.ns - self.nRHS - 1)
            self.MSE = (self.SST - self.SSE) / (self.nobs - self.nRHS - 1)
    
            
            invR = np.linalg.inv(R)
            self.StdErr = np.sqrt(np.diag(invR @ invR.T * self.MSE))
            self.tValues = np.array(self.Coef) / self.StdErr
            self.pValues = 2 * (1 - stats.t.cdf(np.abs(self.tValues), df=self.nobs - self.nRHS - 1))
            self.ConfInt = [
                    (coef - 1.96 * stderr, coef + 1.96 * stderr)
                    for coef, stderr in zip(self.Coef, self.StdErr)
            ]
        except Exception as e:
            raise ValueError(f"An error occurred during fitting model: {e}")

    def predict(self,x_pred = None): #若没传入x_pred 则默认pred x_test的值
        if x_pred == None:
            if self.train_test == False:
                raise ValueError("Your model has not test value")
            else:
                self.X_p = self.X_test
        else:
            if x_pred.shape[1]==self.X:
                self.X_p = x_pred
            else:
                raise ValueError("The number of columns in X does not match the model.")
                
        ns = self.X_p.shape[0]
        self.X_p = np.hstack([np.ones((ns, 1)), self.X_p])
        self.y_pred = self.getYhat(pd.DataFrame(self.X_p,columns = self.RHS),self.Coef)
        return self.y_pred

    def evaluate(self,y_predict,y_true):
        try:
            self.MAE = mean_absolute_error(y_true, y_predict)
            self.MSE = mean_squared_error(y_true,y_predict)
            self.r2 = r2_score(y_true, y_predict)
        except Exception as e:
            return f"An error occurred while evaluating model: {e}"
        return self.MSE,self.MAE,self.r2                    
    
    def __repr__(self):
        try:
            s = '--------------------OLS -----------------------\n'
            s += 'nObs'.ljust(10) + str(self.nobs).rjust(10) + '    '
            s += 'nRegressor'.ljust(10) + str(self.nRHS).rjust(10) + '\n'
            s += 'SST'.ljust(10) + ('%.3f' % self.SST).rjust(10) + '    '
            s += 'SSE'.ljust(10) + ('%.3f' % self.SSE).rjust(10) + '\n'
            s += 'R2'.ljust(10) + ('%.3f' % self.R2).rjust(10) + '    '
            s += 'adjR2'.ljust(10) + ('%.3f' % self.adjR2).rjust(10) + '\n'
            s += 'MSE'.ljust(10) + ('%.3f' % self.MSE).rjust(10) + '    '
            s += 'RMSE'.ljust(10) + ('%.3f' % np.sqrt(self.MSE)).rjust(10) + '\n'
            s += '-----------------------------------------------\n'
            s += 'Variable       Coef        StdErr      tValue      P>|t|      [0.025      0.975]\n'
            for v, beta, stderr, tval, pval, ci in zip(self.RHS, self.Coef, self.StdErr, self.tValues, self.pValues, self.ConfInt):
                s += f"{v.ljust(14)}{beta:10.4f}{stderr:12.4f}{tval:12.4f}{pval:12.4f}{ci[0]:12.4f}{ci[1]:12.4f}\n"
            s += '----------------------------\n'
            return s
        except Exception as e:
            return f"An error occurred while generating the string representation: {e}"

In [11]:
#影响特征分析
if __name__ == "__main__":
    lm=LinReg('Student_performance_data _.csv',11,[0,1,2,3,4,5,6,7,8,9,10],False)
    lm.fit()
    print(lm)

--------------------OLS -----------------------
nObs            2392    nRegressor        11
SST         2002.487    SSE         1910.415
R2             0.954    adjR2          0.954
MSE            0.039    RMSE           0.197
-----------------------------------------------
Variable       Coef        StdErr      tValue      P>|t|      [0.025      0.975]
Intercept         2.5064      0.0164    152.4742      0.0000      2.4742      2.5386
Gender            0.0144      0.0081      1.7865      0.0742     -0.0014      0.0302
Ethnicity         0.0028      0.0039      0.7187      0.4724     -0.0049      0.0105
ParentalEducation    0.0016      0.0040      0.3903      0.6964     -0.0063      0.0095
StudyTimeWeekly    0.0289      0.0007     40.5719      0.0000      0.0275      0.0303
Absences         -0.0998      0.0005   -209.5087      0.0000     -0.1007     -0.0988
Tutoring          0.2503      0.0088     28.4816      0.0000      0.2331      0.2675
ParentalSupport    0.1513      0.0036     42

# 神经网络

In [14]:
class Neural:
    def __init__(self,dataFn,iy,iXs, hidden_units1=4, hidden_units2=2, learning_rate=0.01):
        
        try:
            self.df = pd.read_csv(dataFn)
        except FileNotFoundError:
            raise FileNotFoundError(f"File {dataFn} not found.")
        except Exception as e:
            raise ValueError(f"An error occurred while loading the data: {e}")

        
        try:
            if iy in iXs:
                raise ValueError("y variable found in the x list.")
            
            if not isinstance(iy, (int, str)):
                raise TypeError("iy should be either an integer or string representing the dependent variable.")

            if isinstance(iy, int):
                iy = [iy]
        except Exception as e:
            raise ValueError(f"An error occurred during match x and y: {e}")

        self.X = self.df.iloc[:,list(iXs)]
        self.y = self.df.iloc[:,list(iy)].squeeze()
        
        self.model = Sequential()
        self.model.add(Dense(hidden_units1, activation='relu'))
        self.model.add(Dense(hidden_units2, activation='relu'))
        self.model.add(Dense(1)) 
        self.compile_model(learning_rate)

    
    def compile_model(self, learning_rate):
        self.model.compile(optimizer=Adam(learning_rate=learning_rate),
                           loss='mean_squared_error',
                           metrics=['mae'])
 
    def fit(self, epochs=30, batch_size=32, validation_split=0.3):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size=validation_split, random_state=42)
        history = self.model.fit(self.X_train, self.y_train, epochs=epochs, batch_size=batch_size,
                                 validation_data=(self.X_test, self.y_test), verbose=0)
        return history
 
    def evaluate(self, y_true, y_predict):
        try:
            self.MAE = mean_absolute_error(y_true, y_predict)
            self.MSE = mean_squared_error(y_true,y_predict)
            self.r2 = r2_score(y_true, y_predict)
        except Exception as e:
            return f"An error occurred while evaluating model: {e}"
        return self.MSE,self.MAE,self.r2 
 
    def predict(self, x_pred=None):
        if x_pred == None:
                self.X_p = self.X_test
        else:
            if x_pred.shape[1]==self.X:
                self.X_p = x_pred
            else:
                raise ValueError("The number of columns in X does not match the model.")
        return self.model.predict(self.X_p)

In [24]:
learning_rate_list = [1.0,0.1,0.01,0.001,0.0001,0.00001] #寻找最优的学习率
if __name__ == "__main__":
    best_mse = 10e5
    for i in learning_rate_list:
        model = Neural('Student_performance_data _.csv',11,[0,1,2,3,4,5,6,7,8,9,10],hidden_units1=4, hidden_units2=2, learning_rate=i)
        history = model.fit()
        y_pred = model.predict()
        mse,mae,r2 = model.evaluate(y_pred,model.y_test)
        print(mse)
        if mse<best_mse:
            best_mse = mse
            best_learning_rate1 = i
    print(best_learning_rate1)

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
0.9040418418691031
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
0.07492129516665537
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
0.04144883594452399
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
0.04785906402783564
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
26.989167277173113
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
6.769633019913292
0.01


In [28]:
hidden_list = [(32,16),(16,8),(8,4),(4,2),(2,1)]
if __name__ == "__main__":
    best_mse = 10e5
    for (i,j) in hidden_list:
        model = Neural('Student_performance_data _.csv',11,[0,1,2,3,4,5,6,7,8,9,10],hidden_units1=i, hidden_units2=j, learning_rate=best_learning_rate1)
        history = model.fit()
        y_pred = model.predict()
        mse,mae,r2 = model.evaluate(y_pred,model.y_test)
        print(mse)
        if mse<best_mse:
            best_mse = mse
            best_hidden = (i,j)
    print(best_hidden)

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
0.04529761902693045
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
0.04009385219261913
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
0.05077240094334294
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
0.03873391120974965
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
0.8426069566283896
(4, 2)


# Catboost Regression

In [32]:
class CatBoostRegression:
    def __init__(self,dataFn,iy,iXs, iterations=1500, learning_rate=0.01, depth=3, loss_function='RMSE',verbose=200):
        try:
            self.df = pd.read_csv(dataFn)
        except FileNotFoundError:
            raise FileNotFoundError(f"File {dataFn} not found.")
        except Exception as e:
            raise ValueError(f"An error occurred while loading the data: {e}")

        
        try:
            if iy in iXs:
                raise ValueError("y variable found in the x list.")
            
            if not isinstance(iy, (int, str)):
                raise TypeError("iy should be either an integer or string representing the dependent variable.")

            if isinstance(iy, int):
                iy = [iy]
        except Exception as e:
            raise ValueError(f"An error occurred during match x and y: {e}")

        self.X = self.df.iloc[:,list(iXs)]
        self.y = self.df.iloc[:,list(iy)].squeeze()
        self.model = CatBoostRegressor(
            iterations=iterations,
            learning_rate=learning_rate,
            depth=depth,
            loss_function=loss_function,
            verbose=verbose
        )

    def fit(self, validation_split=0.3, random_state=42):


        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size=validation_split, random_state=random_state)

        train_pool = Pool(self.X_train, self.y_train)
        test_pool = Pool(self.X_test, self.y_test)


        self.model.fit(train_pool, eval_set=test_pool, early_stopping_rounds=50)
        
    def predict(self, x_pred=None):
        if x_pred == None:
            self.X_p = self.X_test
        else:
            if x_pred.shape[1]==self.X:
                self.X_p = x_pred
            else:
                raise ValueError("The number of columns in X does not match the model.")
        return self.model.predict(self.X_p)

    def evaluate(self, y_true, y_predict):
        try:
            self.MAE = mean_absolute_error(y_true, y_predict)
            self.MSE = mean_squared_error(y_true,y_predict)
            self.r2 = r2_score(y_true, y_predict)
        except Exception as e:
            return f"An error occurred while evaluating model: {e}"
        return self.MSE,self.MAE,self.r2 

In [50]:
learning_rate_list = [1.0,0.1,0.01,0.001,0.0001,0.00001] #寻找最优的学习率
if __name__ == "__main__":
    best_mse = 10e5
    for i in learning_rate_list:
        model = CatBoostRegression('Student_performance_data _.csv',11,[0,1,2,3,4,5,6,7,8,9,10],iterations=1500,learning_rate=i,verbose = 400)
        history = model.fit()
        y_pred = model.predict()
        mse,mae,r2 = model.evaluate(y_pred,model.y_test)
        print(mse)
        if mse<best_mse:
            best_mse = mse
            best_learning_rate2 = i
    print(best_learning_rate2)

0:	learn: 0.4818167	test: 0.4789722	best: 0.4789722 (0)	total: 513us	remaining: 769ms
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.2032523216
bestIteration = 17

Shrink model to first 18 iterations.
0.04131150687489733
0:	learn: 0.8495600	test: 0.8540266	best: 0.8540266 (0)	total: 644us	remaining: 966ms
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.1967268565
bestIteration = 152

Shrink model to first 153 iterations.
0.03870145674503466
0:	learn: 0.9072264	test: 0.9114523	best: 0.9114523 (0)	total: 582us	remaining: 873ms
400:	learn: 0.2467937	test: 0.2509338	best: 0.2509338 (400)	total: 164ms	remaining: 450ms
800:	learn: 0.1995759	test: 0.2043584	best: 0.2043584 (800)	total: 356ms	remaining: 311ms
1200:	learn: 0.1902117	test: 0.1982288	best: 0.1982066 (1195)	total: 555ms	remaining: 138ms
1499:	learn: 0.1872391	test: 0.1976859	best: 0.1976444 (1459)	total: 710ms	remaining: 0us

bestTest = 0.1976444096
bestIteration = 1459

Shrink model to fir

In [48]:
depth_list = [2,3,4,5,6,7,8,9,10] #寻找最优的深度
if __name__ == "__main__":
    best_mse = 10e5
    for i in depth_list:
        model = CatBoostRegression('Student_performance_data _.csv',11,[0,1,2,3,4,5,6,7,8,9,10],iterations=1500,learning_rate=0.1,depth = i,verbose = 400)
        history = model.fit()
        y_pred = model.predict()
        mse,mae,r2 = model.evaluate(y_pred,model.y_test)
        print(mse)
        if mse<best_mse:
            best_mse = mse
            best_depth = i
    print(best_depth)

0:	learn: 0.8553460	test: 0.8599660	best: 0.8599660 (0)	total: 424us	remaining: 637ms
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.1966341271
bestIteration = 236

Shrink model to first 237 iterations.
0.038664980641318
0:	learn: 0.8495600	test: 0.8540266	best: 0.8540266 (0)	total: 481us	remaining: 722ms
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.1967268565
bestIteration = 152

Shrink model to first 153 iterations.
0.03870145674503466
0:	learn: 0.8497681	test: 0.8542653	best: 0.8542653 (0)	total: 698us	remaining: 1.05s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.1982242694
bestIteration = 207

Shrink model to first 208 iterations.
0.03929286173703176
0:	learn: 0.8501308	test: 0.8550764	best: 0.8550764 (0)	total: 744us	remaining: 1.12s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.19917309
bestIteration = 105

Shrink model to first 106 iterations.
0.0396699203336386
0:	learn: 0.8438789	test: 0.8

# 模型评估

In [52]:
def evaluate_model(model,to_print = False): #返回mae mse r2
    history = model.fit()
    
    y_pred = model.predict()
    
    mse, mae, r2 = model.evaluate(y_pred, model.y_test)
    
    # Print and return the metrics
    if to_print:
        print(f'Test Mean Squared Error (MSE): {mse}')
        print(f'Test Mean Absolute Error (MAE): {mae}')
        print(f'Test R^2: {r2}')
        
    return {"MAE": mae, "MSE": mse, "R2": r2}

In [54]:
def plot_metrics_comparison(metrics_list, model_names, save_path=None):

    metric_categories = ['MSE', 'MAE', 'R2']
    num_models = len(metrics_list)
    
    x = np.arange(len(metric_categories)) 
    width = 0.25 
    offsets = [(i - (num_models - 1) / 2) * width for i in range(num_models)]
    

    plt.figure(figsize=(10, 6))
    for i, metrics in enumerate(metrics_list):
        values = [metrics[metric] for metric in metric_categories]
        plt.bar(x + offsets[i], values, width, label=model_names[i])
    

    for i, metrics in enumerate(metrics_list):
        values = [metrics[metric] for metric in metric_categories]
        for j, value in enumerate(values):
            plt.text(x[j] + offsets[i], value + 0.01, f"{value:.4f}", ha='center', va='bottom', fontsize=10)
    
    # Add labels, title, legend
    plt.xticks(x, metric_categories, fontsize=12)
    plt.ylabel("Score", fontsize=12)
    plt.title("Comparison of Model Performance Metrics", fontsize=16)
    plt.legend(title="Models", fontsize=10)
    plt.ylim(0, max(max(metrics.values()) for metrics in metrics_list) * 1.2)  # Adjust y-axis range

    # Save or show plot
    if save_path:
        plt.savefig(save_path, format='png', dpi=300, bbox_inches='tight')
    else:
        plt.show()
    plt.close()

In [56]:
def evaluate_and_compare(models, model_names, save_path=None):

    metrics_list = []
    for model in models:
        metrics = evaluate_model(model)
        metrics_list.append(metrics)
    
    # Plot comparison
    plot_metrics_comparison(metrics_list, model_names, save_path)

In [58]:
if __name__ == "__main__":
    model1 = LinReg('Student_performance_data _.csv', 11, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], True)
    results1 = evaluate_model(model1)
    print("Linear Regression Results:", results1)

    model2 = Neural('Student_performance_data _.csv',11,[0,1,2,3,4,5,6,7,8,9,10],hidden_units1=best_hidden[0], hidden_units2=best_hidden[1], learning_rate=best_learning_rate1)
    results2 = evaluate_model(model2)
    print("Neural Network Results:", results2)

    # CatBoost Regression Model
    model3 = CatBoostRegression('Student_performance_data _.csv', 11, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], learning_rate=best_learning_rate2,depth = best_depth)
    results3 = evaluate_model(model3)
    print("CatBoost Regression Results:", results3)
    model_names = ["Linear Regression","Neural Network", "CatBoost Regression"]
    # Evaluate and compare
    evaluate_and_compare([model1, model2, model3], model_names, save_path="model_comparison.png")

Linear Regression Results: {'MAE': 0.15608422447078696, 'MSE': 0.03716551397308837, 'R2': 0.9568235795639569}
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Neural Network Results: {'MAE': 0.15737932192378332, 'MSE': 0.03821326716544601, 'R2': 0.9506603109646985}
0:	learn: 0.8553460	test: 0.8599660	best: 0.8599660 (0)	total: 862us	remaining: 1.29s
200:	learn: 0.1927475	test: 0.1971580	best: 0.1971514 (192)	total: 70.2ms	remaining: 454ms
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.1966341271
bestIteration = 236

Shrink model to first 237 iterations.
CatBoost Regression Results: {'MAE': 0.1566809318291144, 'MSE': 0.038664980641318, 'R2': 0.9506808711047018}
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
0:	learn: 0.8553460	test: 0.8599660	best: 0.8599660 (0)	total: 426us	remaining: 639ms
200:	learn: 0.1927475	test: 0.1971580	best: 0.1971514 (192)	total: 70.4ms	remaining: 455ms
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.1966341271
bestIteration = 236

Sh